## Homework
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

- 1054112
- **1154112** <- answer
- 1254112
- 1354112

In [2]:
df = pd.read_parquet('data/fhv_tripdata_2021-01.parquet')

In [3]:
df.shape

(1154112, 7)

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?

- 15.16
- **19.16** <- answer
- 24.16
- 29.16

In [4]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,NaN,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,NaN,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,NaN,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,NaN,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,NaN,B00037


In [5]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
df['dropOff_datetime'] = pd.to_datetime(df['dropOff_datetime'], format='%Y-%m-%d %H:%M:%S')
    
df['duration'] = (df['dropOff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60 

In [6]:
round(df['duration'].mean(), 2)

19.17

## Data preparation

Check the distribution of the duration variable. There are some outliers.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?

In [7]:
df['duration'].describe().round(2)

count    1154112.00
mean          19.17
std          398.69
min            0.02
25%            7.77
50%           13.40
75%           22.28
max       423371.05
Name: duration, dtype: float64

In [8]:
df2 = df[(df['duration'] >= 1) & (df['duration'] <=60)]
print('N of records we dropped =', len(df) - len(df2))

N of records we dropped = 44286


## Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

- 53%
- 63%
- 73%
- **83%** <- answer

In [20]:
df2['PUlocationID'] = df2['PUlocationID'].fillna('-1').astype(str)
df2['DOlocationID'] = df2['DOlocationID'].fillna('-1').astype(str)
(df2['PUlocationID'].value_counts(normalize=True)*100).round(2)

-1       83.53
221.0     0.75
206.0     0.61
129.0     0.48
115.0     0.37
         ...  
111.0     0.00
27.0      0.00
34.0      0.00
2.0       0.00
110.0     0.00
Name: PUlocationID, Length: 262, dtype: float64

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix? (The number of columns).

- 2
- 152
- 352
- **525** <- answer
- 725

In [10]:
from sklearn.feature_extraction import DictVectorizer

In [11]:
categorical = ['PUlocationID', 'DOlocationID']

dv = DictVectorizer()

train_dicts = df2[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [12]:
X_train.toarray().shape

(1109826, 525)

In [13]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 5.52
- **10.52** <- answer
- 15.52
- 20.52

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
target = 'duration'
y_train = df2[target]

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107205451

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

- 6.01
- **11.01** <- answer
- 16.01
- 21.01

In [18]:
df = pd.read_parquet('data/fhv_tripdata_2021-02.parquet')
    
df['duration'] = (
    pd.to_datetime(df['dropOff_datetime'], format='%Y-%m-%d %H:%M:%S') - \
    pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
).dt.total_seconds() / 60

df = df[(df['duration'] >= 1) & (df['duration'] <=60)]

df['PUlocationID'] = df['PUlocationID'].fillna('-1').astype(str)
df['DOlocationID'] = df['DOlocationID'].fillna('-1').astype(str)

categorical = ['PUlocationID', 'DOlocationID']
X_val = dv.transform(df[categorical].to_dict(orient='records'))

target = 'duration'
y_val = df[target]

mean_squared_error(y_val, lr.predict(X_val), squared=False)

11.014283139629091